# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f0b0c579f70>
├── 'a' --> tensor([[ 0.5862,  0.4277,  0.2169],
│                   [-0.2527, -1.0134,  2.3659]])
└── 'x' --> <FastTreeValue 0x7f0b0c513130>
    └── 'c' --> tensor([[ 1.2291,  1.2786,  0.3704, -1.8820],
                        [ 0.8335,  0.3351, -0.8345, -0.3564],
                        [ 0.3241,  1.5688, -0.6366,  1.2889]])

In [4]:
t.a

tensor([[ 0.5862,  0.4277,  0.2169],
        [-0.2527, -1.0134,  2.3659]])

In [5]:
%timeit t.a

67 ns ± 0.0423 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f0b0c579f70>
├── 'a' --> tensor([[-0.3773,  0.4522, -0.0147],
│                   [-0.7168,  0.5549,  0.3681]])
└── 'x' --> <FastTreeValue 0x7f0b0c513130>
    └── 'c' --> tensor([[ 1.2291,  1.2786,  0.3704, -1.8820],
                        [ 0.8335,  0.3351, -0.8345, -0.3564],
                        [ 0.3241,  1.5688, -0.6366,  1.2889]])

In [7]:
%timeit t.a = new_value

70.5 ns ± 0.0587 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5862,  0.4277,  0.2169],
               [-0.2527, -1.0134,  2.3659]]),
    x: Batch(
           c: tensor([[ 1.2291,  1.2786,  0.3704, -1.8820],
                      [ 0.8335,  0.3351, -0.8345, -0.3564],
                      [ 0.3241,  1.5688, -0.6366,  1.2889]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5862,  0.4277,  0.2169],
        [-0.2527, -1.0134,  2.3659]])

In [11]:
%timeit b.a

55.4 ns ± 0.0215 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7279, -0.2912, -0.1595],
               [-1.4911, -0.5103, -1.6833]]),
    x: Batch(
           c: tensor([[ 1.2291,  1.2786,  0.3704, -1.8820],
                      [ 0.8335,  0.3351, -0.8345, -0.3564],
                      [ 0.3241,  1.5688, -0.6366,  1.2889]]),
       ),
)

In [13]:
%timeit b.a = new_value

481 ns ± 0.274 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

841 ns ± 0.198 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 32.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 442 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 5.68 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f0a61218e50>
├── 'a' --> tensor([[[ 0.5862,  0.4277,  0.2169],
│                    [-0.2527, -1.0134,  2.3659]],
│           
│                   [[ 0.5862,  0.4277,  0.2169],
│                    [-0.2527, -1.0134,  2.3659]],
│           
│                   [[ 0.5862,  0.4277,  0.2169],
│                    [-0.2527, -1.0134,  2.3659]],
│           
│                   [[ 0.5862,  0.4277,  0.2169],
│                    [-0.2527, -1.0134,  2.3659]],
│           
│                   [[ 0.5862,  0.4277,  0.2169],
│                    [-0.2527, -1.0134,  2.3659]],
│           
│                   [[ 0.5862,  0.4277,  0.2169],
│                    [-0.2527, -1.0134,  2.3659]],
│           
│                   [[ 0.5862,  0.4277,  0.2169],
│                    [-0.2527, -1.0134,  2.3659]],
│           
│                   [[ 0.5862,  0.4277,  0.2169],
│                    [-0.2527, -1.0134,  2.3659]]])
└── 'x' --> <FastTreeValue 0x7f0b0c58df70>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 51.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f0a611fca30>
├── 'a' --> tensor([[ 0.5862,  0.4277,  0.2169],
│                   [-0.2527, -1.0134,  2.3659],
│                   [ 0.5862,  0.4277,  0.2169],
│                   [-0.2527, -1.0134,  2.3659],
│                   [ 0.5862,  0.4277,  0.2169],
│                   [-0.2527, -1.0134,  2.3659],
│                   [ 0.5862,  0.4277,  0.2169],
│                   [-0.2527, -1.0134,  2.3659],
│                   [ 0.5862,  0.4277,  0.2169],
│                   [-0.2527, -1.0134,  2.3659],
│                   [ 0.5862,  0.4277,  0.2169],
│                   [-0.2527, -1.0134,  2.3659],
│                   [ 0.5862,  0.4277,  0.2169],
│                   [-0.2527, -1.0134,  2.3659],
│                   [ 0.5862,  0.4277,  0.2169],
│                   [-0.2527, -1.0134,  2.3659]])
└── 'x' --> <FastTreeValue 0x7f0a611fc2b0>
    └── 'c' --> tensor([[ 1.2291,  1.2786,  0.3704, -1.8820],
                        [ 0.8335,  0.3351, -0.8345, -0.3564],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 134 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.7 µs ± 46.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.2291,  1.2786,  0.3704, -1.8820],
                       [ 0.8335,  0.3351, -0.8345, -0.3564],
                       [ 0.3241,  1.5688, -0.6366,  1.2889]],
              
                      [[ 1.2291,  1.2786,  0.3704, -1.8820],
                       [ 0.8335,  0.3351, -0.8345, -0.3564],
                       [ 0.3241,  1.5688, -0.6366,  1.2889]],
              
                      [[ 1.2291,  1.2786,  0.3704, -1.8820],
                       [ 0.8335,  0.3351, -0.8345, -0.3564],
                       [ 0.3241,  1.5688, -0.6366,  1.2889]],
              
                      [[ 1.2291,  1.2786,  0.3704, -1.8820],
                       [ 0.8335,  0.3351, -0.8345, -0.3564],
                       [ 0.3241,  1.5688, -0.6366,  1.2889]],
              
                      [[ 1.2291,  1.2786,  0.3704, -1.8820],
                       [ 0.8335,  0.3351, -0.8345, -0.3564],
                       [ 0.3241,  1.5688, -0.6366,  1.2889]],

In [26]:
%timeit Batch.stack(batches)

77.4 µs ± 100 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.2291,  1.2786,  0.3704, -1.8820],
                      [ 0.8335,  0.3351, -0.8345, -0.3564],
                      [ 0.3241,  1.5688, -0.6366,  1.2889],
                      [ 1.2291,  1.2786,  0.3704, -1.8820],
                      [ 0.8335,  0.3351, -0.8345, -0.3564],
                      [ 0.3241,  1.5688, -0.6366,  1.2889],
                      [ 1.2291,  1.2786,  0.3704, -1.8820],
                      [ 0.8335,  0.3351, -0.8345, -0.3564],
                      [ 0.3241,  1.5688, -0.6366,  1.2889],
                      [ 1.2291,  1.2786,  0.3704, -1.8820],
                      [ 0.8335,  0.3351, -0.8345, -0.3564],
                      [ 0.3241,  1.5688, -0.6366,  1.2889],
                      [ 1.2291,  1.2786,  0.3704, -1.8820],
                      [ 0.8335,  0.3351, -0.8345, -0.3564],
                      [ 0.3241,  1.5688, -0.6366,  1.2889],
                      [ 1.2291,  1.2786,  0.3704, -1.8820],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 187 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

311 µs ± 8.24 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
